In [ ]:
#%pip uninstall TabularExperimentTrackerClient --y
#%pip uninstall NeuralNetworksTrainingPackage --y
%conda install python=3.11 --yes
%conda install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage
%conda install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient

usage: conda [-h] [-V] command ...
conda: error: unrecognized arguments: --upgrade

Note: you may need to restart the kernel to use updated packages.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - //github.com/bartosz-g/neuralnetworkstrainingpackage

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.

In [3]:
%pip install ./models/ltr/data-scienc-ml-t3-medium/random_forestry-0.10.0-cp311-cp311-linux_x86_64.whl

ERROR: random_forestry-0.10.0-cp311-cp311-linux_x86_64.whl is not a supported wheel on this platform.
Note: you may need to restart the kernel to use updated packages.


In [13]:
!python --version

Python 3.10.6


In [14]:
!uname -m

x86_64


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import time

In [ ]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../creds/'
creds_orch_file = "creds-orch.txt"
creds_openml_file = "creds-openml.txt"



with open(os.path.join(path, creds_orch_file), 'r') as file:
    lines = file.readlines()
    orchname = lines[0].strip()
    orchsecret = lines[1].strip()

with open (os.path.join(path, creds_openml_file), "r") as myfile:
    openMLAPIKey = myfile.read()

ex = ExperimentClient(verbose = True)


ex.define_orch_cred(orchname, orchsecret)
ex.define_opml_cred(openMLAPIKey)

# 1. Data pre-processing

In [ ]:
from NeuralNetworksTrainingPackage.event_handler import dataPreProcessingEventEmitter
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import filterCardinality, quantileTransform, trunctuateData
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import oneHotEncodePredictors, oneHotEncodeTargets, toDataFrame

n_sample = 20000
quantile_transform_distribution='uniform'


data_pre_processing = dataPreProcessingEventEmitter()

filter_cardinality = filterCardinality()
data_pre_processing.add_pre_processing('regression', filter_cardinality)
data_pre_processing.add_pre_processing('classification', filter_cardinality)

quantile_transform = quantileTransform(output_distribution = quantile_transform_distribution)
data_pre_processing.add_pre_processing('regression', quantile_transform)
data_pre_processing.add_pre_processing('classification', quantile_transform)

trunctuate_data = trunctuateData(n = n_sample)
data_pre_processing.add_pre_processing('regression', trunctuate_data)
data_pre_processing.add_pre_processing('classification', trunctuate_data)

one_hot_encode_predictors = oneHotEncodePredictors()
data_pre_processing.add_pre_processing('regression', one_hot_encode_predictors)
data_pre_processing.add_pre_processing('classification', one_hot_encode_predictors)

one_hot_encode_targets = oneHotEncodeTargets()
data_pre_processing.add_pre_processing('classification', one_hot_encode_targets)

to_data_frame = toDataFrame()
data_pre_processing.add_pre_processing('regression', to_data_frame)
data_pre_processing.add_pre_processing('classification', to_data_frame)